In [1]:
import numpy as np
import tensorflow as tf
import os
from PIL import Image

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,BatchNormalization,PReLU
from tensorflow.keras.preprocessing.image import img_to_array, load_img,ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

2024-04-07 19:26:10.674642: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 19:26:10.674707: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 19:26:10.676230: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
class_labels = os.listdir('/kaggle/input/brain-tumor-dataset/Training')
class_labels

['pituitary', 'notumor', 'meningioma', 'glioma']

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('/kaggle/input/brain-tumor-dataset/Training', 
                                                 target_size = (256, 256), 
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                                 color_mode = 'grayscale',
                                                 shuffle = True)

Found 5712 images belonging to 4 classes.


In [4]:
type(training_set)

keras.src.legacy.preprocessing.image.DirectoryIterator

## Shalow Arch

In [5]:
input_shape= (256,256,1)
model = Sequential([
    
    Conv2D(16,(5,5), activation = 'leaky_relu', padding ='same', input_shape = input_shape),
    
    MaxPooling2D((2,2)),
    
    Conv2D(32,(5,5), activation = 'leaky_relu', padding ='same'),
    #Conv2D(16,(5,5), activation = 'leaky_relu', padding ='same'),

    MaxPooling2D((2,2)),
    
    
    Conv2D(32,(5,5), activation = 'leaky_relu', padding='same'),

    MaxPooling2D((2,2)),
    # Conv2D(52,(5,5),activation = 'leaky_relu', padding = 'same', strides =(2,2)),
    
    Conv2D(64,(5,5), activation = 'leaky_relu', padding = 'same'),
    # Conv2D(64,(5,5), activation = 'leaky_relu', padding = 'same', strides = (2,2)),
    MaxPooling2D((2,2)),
    
    Flatten(),
    BatchNormalization(),
    Dense(32, kernel_regularizer = regularizers.l2(0.015)),
    PReLU(),
    #Dropout(0.5),
    Dense(32, kernel_regularizer = regularizers.l2(0.015)),
    PReLU(),
    # Dropout(0.5),
    Dense(4 , activation = 'softmax')
])

model.compile(optimizer= Adam(learning_rate=1e-5), loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 16)   │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 32)   │        12,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 32)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16384)          │        65,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │       524,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu (PReLU)                 │ (None, 32)             │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_1 (PReLU)               │ (None, 32)             │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 681,252 (2.60 MB)

 Trainable params: 648,484 (2.47 MB)

 Non-trainable params: 32,768 (128.00 KB)

In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/kaggle/input/brain-tumor-dataset/Testing',
                                            target_size = (256, 256),
                                            batch_size = 16,
                                            class_mode = 'categorical',
                                            color_mode = 'grayscale',
                                            shuffle = False)

Found 1311 images belonging to 4 classes.


In [7]:
model.fit(training_set,validation_data=test_set,epochs=50)

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  5/357 ━━━━━━━━━━━━━━━━━━━━ 12s 35ms/step - accuracy: 0.1598 - loss: 2.9587

I0000 00:00:1712517983.403600    1128 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1712517983.423590    1128 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


354/357 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5381 - loss: 2.5294

W0000 00:00:1712517997.113971    1129 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


357/357 ━━━━━━━━━━━━━━━━━━━━ 25s 48ms/step - accuracy: 0.5393 - loss: 2.5272 - val_accuracy: 0.6590 - val_loss: 2.4456
Epoch 2/50
357/357 ━━━━━━━━━━━━━━━━━━━━ 16s 44ms/step - accuracy: 0.7671 - loss: 2.0311 - val_accuracy: 0.7773 - val_loss: 1.9668
Epoch 3/50
357/357 ━━━━━━━━━━━━━━━━━━━━ 16s 44ms/step - accuracy: 0.8002 - loss: 1.8863 - val_accuracy: 0.7841 - val_loss: 1.8505
Epoch 4/50
357/357 ━━━━━━━━━━━━━━━━━━━━ 16s 43ms/step - accuracy: 0.8282 - loss: 1.7829 - val_accuracy: 0.8337 - val_loss: 1.7341
Epoch 5/50
357/357 ━━━━━━━━━━━━━━━━━━━━ 16s 45ms/step - accuracy: 0.8627 - loss: 1.6919 - val_accuracy: 0.8581 - val_loss: 1.6492
Epoch 6/50
357/357 ━━━━━━━━━━━━━━━━━━━━ 17s 46ms/step - accuracy: 0.8721 - loss: 1.6252 - val_accuracy: 0.8513 - val_loss: 1.6145
Epoch 7/50
357/357 ━━━━━━━━━━━━━━━━━━━━ 16s 44ms/step - accuracy: 0.8823 - loss: 1.5608 - val_accuracy: 0.8772 - val_loss: 1.5462
Epoch 8/50
357/357 ━━━━━━━━━━━━━━━━━━━━ 16s 45ms/step - accuracy: 0.8996 - loss: 1.5000 - val_accurac

In [8]:
y_pred = model.predict(test_set)
y_pred

 6/82 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step

W0000 00:00:1712518786.832406    1126 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


82/82 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step


array([[9.29229140e-01, 3.39276530e-03, 5.73022030e-02, 1.00758225e-02],
       [9.70596671e-01, 1.28868513e-03, 3.36613297e-03, 2.47485023e-02],
       [9.95987952e-01, 2.10504164e-03, 1.40645541e-03, 5.00497816e-04],
       ...,
       [1.31477148e-03, 2.98896758e-03, 5.37963584e-04, 9.95158374e-01],
       [2.25672545e-03, 1.97525573e-04, 4.23891266e-04, 9.97121751e-01],
       [5.09086647e-04, 1.47718063e-03, 2.23707361e-03, 9.95776653e-01]],
      dtype=float32)

In [9]:
y_pred_new = 1*(y_pred > 0.5)
y_pred_new

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       ...,
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1]])

In [ ]:
from keras.preprocessing import image
import numpy as np
test_image = image.load_img('Dataset/Demo/Te-me_0016.jpg', target_size = (512, 512))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = model.predict(test_image)
print(result)
print(training_set.class_indices)
if result[0][0] == 1:
    prediction = 'glioma'
elif result[0][1] == 1:
    prediction = 'meningioma'
elif result[0][2] == 1:
    prediction = 'notumor'
else:
    prediction = 'pituitary'
print(prediction)

In [24]:
# Assuming you have a test_set object (e.g., a Keras ImageDataGenerator flow_from_directory)
# Get the true labels directly from the test_set
y_test_set = test_set.labels

# Convert the data type to integer
y_test_set = y_test_set.astype(int)

# Reshape the array to have shape (num_samples, 1)
y_test_set = y_test_set.reshape(len(y_test_set), 1)
y_test_set = to_categorical(y_test_set)
# Print the resulting y_test_set
print(y_test_set)


[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [16]:
y_pred = model.predict(test_set)
y_pred = (y_pred > 0.5)

82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step


In [17]:
y_pred_round = y_pred.astype(int)
print(y_pred_round)

[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 ...
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]


In [25]:
from sklearn.metrics import confusion_matrix,accuracy_score
import numpy as np

# Assuming y_test and y_pred are your true and predicted labels respectively
y_test_decoded = np.argmax(y_test_set, axis=1)
y_pred_decoded = np.argmax(y_pred_round, axis=1)

cm = confusion_matrix(y_test_decoded, y_pred_decoded)
acc= accuracy_score(y_test_set, y_pred_new)
print(acc,"\n")
print(cm)


0.969488939740656 

[[284  16   0   0]
 [ 14 284   5   3]
 [  0   0 405   0]
 [  0   1   0 299]]
